![jupyter](./pic1.jpg)

![jupyter](./pic2.jpg)

In [1]:
import pandas as pd
df_osdg = pd.read_excel('E:\PostGraduate\OSDG\osdg-new.xlsx')
df_osdg = df_osdg.query('agreement >= .6 and labels_positive > labels_negative').copy()
df_osdg.reset_index(inplace = True)
df_osdg.groupby('sdg', as_index = False).agg(count = ('text_id', 'count'))

,sdg,count
0,1,1241
1,2,830
2,3,1849
3,4,2363
4,5,2433
5,6,1336
6,7,1912
7,8,902
8,9,701
9,10,476


In [2]:
index=[] #store indexes of different classes of sdgs
sample_numbers=[] #store the total numbers of every class of sdg
index.append([])
sample_numbers.append(0)
for i in range(1,16):
    cur_index=df_osdg[df_osdg["sdg"]==i]._stat_axis.values.tolist() 
    index.append(cur_index)
    sample_numbers.append(len(cur_index))
index[5]
sample_numbers[5]
total_sample_nums=len(df_osdg) 

In [3]:
#give a list, randomly divide into n groups
from random import shuffle
def random_grouping(glist,group_nums):
    shuffle(glist) 
    m = int(len(glist)//group_nums) #average num of samples in each group
    list2 = []
    for i in range(0, len(glist), m):
        if (len(glist[i:i+m])<m): #add the left samples into the last group
            list2[-1].extend(glist[i:i+m])
        else: #create a list to load m samples
            list2.append(glist[i:i+m])
    return list2
    
#demo
glist=["1", "2", "3", "4","5"]
print(random_grouping(glist,2))

[['4', '5'], ['1', '2', '3']]


In [4]:
#input a class of sdg that you want to identify,
#out put how many learners are going to be trained,and different negative samples used in each training
def negative_sampling(sdg):
    learner_nums=(total_sample_nums-sample_numbers[sdg])//sample_numbers[sdg]
    negative_groups=[] #randomly divide each negative class of data into groups, 
                       #so that we can use different batch of data when training.
    for i in range (1,16):
        if i!=sdg: #only group negative dataset
            cur_group=random_grouping(index[i],learner_nums)
            negative_groups.append(cur_group)
    negative_id=[] #decide in each training, what negative data are we going to use
    for i in range(learner_nums):
        cur_negative_id=[]
        for j in negative_groups:
            cur_negative_id.extend(j[i])
        negative_id.append(cur_negative_id) 
    return negative_id,learner_nums

In [5]:
#this time we save the dataset to be used
import os
from tqdm import tqdm
def data_split(sdg): #which sdg do you want to identify
    df_positive=df_osdg[df_osdg["sdg"]==sdg]
    negative_id,learner_nums=negative_sampling(sdg)
    data=[]
    for i in range(learner_nums):
        df_negative=df_osdg.loc[negative_id[i]]
        cur_df=pd.concat([df_positive,df_negative])
        data.append(cur_df)
    #save dataset
    folder="sdg"+str(sdg)
    if not (os.path.exists(folder)):
        os.makedirs(folder) 
    for i in tqdm(range(len(data))):
        data[i].to_excel(folder+"\\learner-"+str(i+1)+".xlsx")

In [6]:
#useage:
#for i in range(1,16):
#    data_split(i)